In [3]:
# Install all required libraries (run this cell only once)
!pip install -U langchain-core langgraph
!pip install -U langchain-openai
!pip install -U langchain-groq
!pip install transformers

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [ ]:
"""
Chatbot with LangChain + Groq (October 2025 version)
Author: Chinta Harichandan
Description:
A simple conversational chatbot built using LangChain Core and LangGraph,
powered by Groq’s open LLMs (gpt-oss or llama3 models).
"""

# ======================
# 1️⃣  Imports and Setup
# ======================
import os
from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage, trim_messages
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph


# =========================
# 2️⃣  API Key Configuration
# =========================
# ⚠️ Important:
#   Do NOT hardcode your API key here if uploading to GitHub.
#   Instead, set it as an environment variable in your system:
#     Windows (CMD):  setx GROQ_API_KEY "your_api_key_here"
#     Linux/Mac:      export GROQ_API_KEY="your_api_key_here"
#
#   Or, if testing locally, uncomment the line below and paste your key (REMOVE before upload):
# os.environ["GROQ_API_KEY"] = "gsk_your_api_key_here"

# Verify key presence
if "GROQ_API_KEY" not in os.environ:
    raise EnvironmentError("Please set the GROQ_API_KEY environment variable before running the chatbot.")


# ==================================
# 3️⃣  Initialize the Chat Model
# ==================================
# Use an active Groq model (as of Oct 2025)
# Recommended: gpt-oss-20B or llama3-70b-8192 if supported in your console
model_name = "openai/gpt-oss-20B"
model = init_chat_model(model_name, model_provider="groq")


# =====================================
# 4️⃣  Define the Chatbot’s Behavior
# =====================================
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful and knowledgeable AI assistant."),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

# Trimming for long conversations (keeps only latest ~512 tokens)
trimmer = trim_messages(
    max_tokens=512,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

# Define how the model is called
def call_model(state: MessagesState):
    trimmed = trimmer.invoke(state["messages"])
    prompt = prompt_template.invoke({"messages": trimmed})
    response = model.invoke(prompt)
    return {"messages": response}


# ==================================
# 5️⃣  Build the Chat Workflow
# ==================================
workflow = StateGraph(state_schema=MessagesState)
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory (conversation history)
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)


# ==================================
# 6️⃣  Chat Functions
# ==================================
def chat_turn(user_input: str, thread_id: str = "main"):
    """
    Perform one chat turn and return AI's text response.
    """
    config = {"configurable": {"thread_id": thread_id}}
    input_messages = [HumanMessage(content=user_input)]
    output = app.invoke({"messages": input_messages}, config)
    ai_msg = output["messages"][-1]
    return ai_msg.content


def chat_stream(user_input: str, thread_id: str = "main"):
    """
    Stream AI response in real-time.
    """
    config = {"configurable": {"thread_id": thread_id}}
    input_messages = [HumanMessage(content=user_input)]
    print("Bot: ", end="", flush=True)
    for chunk, metadata in app.stream(
        {"messages": input_messages}, config, stream_mode="messages"
    ):
        if isinstance(chunk, AIMessage):
            print(chunk.content, end="", flush=True)
    print()


# ==================================
# 7️⃣  Example Usage (Safe for GitHub)
# ==================================
if __name__ == "__main__":
    print(f"✅ Chatbot initialized using model: {model_name}")
    print("Type 'exit' to quit.\n")

    while True:
        user_input = input("You: ")
        if user_input.lower() in {"exit", "quit"}:
            print("👋 Goodbye!")
            break
        response = chat_turn(user_input)
        print("Bot:", response)


None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


✅ Chatbot initialized using model: openai/gpt-oss-20B
Type 'exit' to quit.



You:  hello


Bot: Hello! How can I help you today?


You:  what is your name?


Bot: I’m ChatGPT, your AI assistant. How can I help you today?
